In [ ]:
# pip install selenium

In [ ]:
# pip install webdriver_manager

## Import all necessary libraries

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
import pandas as pd
from pandas.errors import EmptyDataError
import time
import itertools
import re
import os
import sched
from concurrent.futures import ThreadPoolExecutor

## Capture the list of top 2000 companies with most H-1B Fillings
#### Not running in headless mode as we need to manually close ads

In [ ]:
start_time = time.time()
print('Timer started.')

dataframes = {}

# headless option to scrape to bypass advertisement that will popup halfway in the scraping process
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())

# navigate to the URL
driver.get("https://h1bdata.info/")

# navigate to the companies page
companies_link = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/ul/li[3]")
companies_link.click()

# locate the dropdown menu and click on the "More..." link
dropdown_menu = driver.find_element(By.CLASS_NAME, "dropdown-menu")
more_link = dropdown_menu.find_element(By.LINK_TEXT, "More...")
more_link.click()

# wait for the table element to be visible
table_element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/center/div/table")))
print('Company table loaded.')

# convert the table to a pandas dataframe
rows = table_element.find_elements(By.XPATH, "./tbody/tr")
company_data = []

# extract data from each row of the table
for row_element in rows:
    row = {}
    columns = row_element.find_elements(By.XPATH, "./td")
    print(len(columns))  # add this line to see how many columns there are in each row
    if len(columns) < 4:
        continue
    row['Rank'] = columns[0].text
    row['Company Name'] = columns[1].text
    row['H-1B Filings'] = columns[2].text
    row['Average Salary'] = columns[3].text
    print(row)  # add this line to see what the row data looks like
    company_data.append(row)

company = pd.DataFrame(company_data)

# save the dataframe as a CSV
company.to_csv("companies.csv", index=False)

driver.quit()

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time} seconds")

## Scrape the URL link for each individual companies
#### Not running in headless mode either to manually close ads

In [ ]:
URL ="https://h1bdata.info/"

# Create an empty list to store the data
table_for_links = []

# Set the headless option
options = webdriver.ChromeOptions()

# Install and configure the ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

# Navigate to the URL
driver.get("https://h1bdata.info/")

print('First link, https://h1bdata.info/, is working.')

# navigate to the companies page
companies_link = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/ul/li[3]")
companies_link.click()

# locate the dropdown menu and click on the "More..." link
dropdown_menu = driver.find_element(By.CLASS_NAME, "dropdown-menu")
more_link = dropdown_menu.find_element(By.LINK_TEXT, "More...")
more_link.click()

print('Second link, https://h1bdata.info/topcompanies.php, is working.')

# List of potential XPath expressions for the overlay
overlay_xpath_list = [
    "/html/body/div/div/div[1]",
    "/html/body/div/div",
    # Add more XPath expressions if needed
]

# Flag to indicate if the overlay is successfully closed
overlay_closed = False

# Iterate through the list of XPath expressions and try to close the overlay
for overlay_xpath in overlay_xpath_list:
    try:
        overlay = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, overlay_xpath)))
        close_button = WebDriverWait(overlay, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#dismiss-button > div > span")))
        close_button.click()
        print("Overlay closed using XPath expression:", overlay_xpath)
        overlay_closed = True
        break  # Exit the loop once the overlay is closed

    except:
        continue  # Try the next XPath expression if this one fails

# If the overlay is not closed, handle the exception
if not overlay_closed:
    print("Overlay not found. Proceeding without closing.")


# wait for the table element to be visible
table_element = WebDriverWait(driver, 240).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/center/div/table")))
print('Company table loaded.')


for i in range(2, 2002):
    try:
        print(f"loop {i}")
        #start scraping from the second company since the first company，infosys limited, does not load properly
        # Scroll the page to bring the element into view
        element = driver.find_element(By.XPATH, f"/html/body/div[2]/center/div/table/tbody/tr[{i}]/td[2]/a")

        # Get the text referring to the URL link
        link_text = element.text

        # Get the href attribute value
        url_link = element.get_attribute("href")

        # Print the URL link
        print("Link Text:",link_text,"URL link:", url_link)

        # Add the data to the list
        table_for_links.append({'Link Text': link_text, 'URL Link': url_link})
    except NoSuchElementException as e:
        print(f"Element not found for company {i-1}. Skipping to next loop. NoSuchElement")
        continue

# Convert the list into a DataFrame
df = pd.DataFrame(table_for_links)
df.to_csv("company_links.csv")
df.head()


driver.quit()

In [ ]:
table_for_links=pd.read_csv('company_links.csv',index_col=0)
table_for_links.head(5)

## Main Scraper Function
#### Running in headless mode

In [ ]:
def scrape_data(year, link_text, url_link, lazy_load = 120):
    url = f'https://h1bdata.info/index.php?year={year}&em={url_link}'

    try:
        options = Options()
        options.add_argument("--headless")  # Run Chrome in headless mode

        # Install and configure the ChromeDriver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Navigate to the URL
        driver.get(url)

        table_element = None  # Initialize table_element to None

        try:
            # Scroll down to the bottom of the page in steps to trigger lazy loading (if any)
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(180)  # Wait to load page
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # After scrolling to the bottom, go back to top of the page
            driver.execute_script("window.scrollTo(0, 0);")

            # Wait for the table element to be visible
            table_element = WebDriverWait(driver, lazy_load).until(
                EC.visibility_of_element_located((By.XPATH, "//table[@id='myTable']")))
            print(f"Table element is visible for {link_text} in year {year}")
        except TimeoutException:
            print(f"Table element is not visible for {link_text} in year {year} within the timeout.")
            driver.quit()
            return False

        if table_element is not None:
            # Retrieve the rows of the table
            rows = table_element.find_elements(By.XPATH, "./tbody/tr")

            company_data = []
            for row_element in rows:
                columns = row_element.find_elements(By.XPATH, "./td")
                if columns is None or len(columns) < 6:
                    continue
                row_data = {
                    'Employer': columns[0].text,
                    'Job Title': columns[1].text if len(columns) > 1 else '',
                    'Base Salary': columns[2].text if len(columns) > 2 else '',
                    'Location': columns[3].text if len(columns) > 3 else '',
                    'Submit Date': columns[4].text if len(columns) > 4 else '',
                    'Start Date': columns[5].text if len(columns) > 5 else ''
                }
                company_data.append(row_data)

            # Close the ChromeDriver
            driver.quit()

            # Create a DataFrame from the collected data
            df_company = pd.DataFrame(company_data)
            
            # Sanitize the link_text
            sanitized_link_text = ScrapeScheduler.sanitize_filename(link_text)

            # Save the DataFrame as a CSV file with the company name and year as the file name
            filename = f"{sanitized_link_text}_{year}.csv"
            df_company.to_csv(filename, index=False)
            print(f"Saved DataFrame for {link_text} in year {year} as {filename}")

            return True

    except Exception as e:
        print(f"An error occurred for {link_text} in year {year}: {str(e)}")
        return False


#### Define a Scheduler class to scrape the h1b visa website at scheduled times in concurrent processes

In [ ]:
class ScrapeScheduler:
    def __init__(self, table_for_links, year_range):
        self.table_for_links = table_for_links
        self.year_range = year_range
        self.missing_links_years = None
        self.scheduler = sched.scheduler(time.time, time.sleep)
        
    @staticmethod
    def sanitize_filename(filename):
        return re.sub(r'[\\/:"*?<>|]', '', filename) #sanitize special characters for csv filenames

    def get_missing_years_and_links(self, directory):
        link_texts = self.table_for_links['Link Text'].unique()
        all_years = set(self.year_range)

        missing_links_years = []
        for link_text in link_texts:
            files = [f for f in os.listdir(directory) if f.startswith(link_text)]
            found_years = {int(f.split('_')[1].split('.')[0]) for f in files}
            missing_years = all_years - found_years
            for year in missing_years:
                missing_links_years.append({'Link Text': link_text, 'URL Link': self.table_for_links.loc[self.table_for_links['Link Text'] == link_text, 'URL Link'].iloc[0], 'Year': year})

        return pd.DataFrame(missing_links_years)

    def run_scrape_data_with_years(self, mode='normal', lazy_load=120):
        if mode == 'missing':
            self.missing_links_years = self.get_missing_years_and_links(os.getcwd())
        elif mode == 'normal':
            self.missing_links_years = self.table_for_links.copy()
            self.missing_links_years['Year'] = [self.year_range] * len(self.missing_links_years)  # Assign the year range list to each row
            self.missing_links_years = self.missing_links_years.explode('Year')  # Create a new row for each year

        if self.missing_links_years.empty:
            print("No missing years and links to scrape.")
            return

        max_workers = 40
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for _, row in self.missing_links_years.iterrows():
                link_text = row['Link Text']
                url_link = row['URL Link'][44:]
                year = row['Year']
                future = executor.submit(scrape_data, year, link_text, url_link, lazy_load)
                futures.append((future, link_text, year))

            for future, link_text, year in futures:
                try:
                    if future.result():
                        # If scraping was successful, remove this row from the DataFrame
                        self.missing_links_years = self.missing_links_years[self.missing_links_years['Link Text'] != link_text]
                    else:
                        print(f"Failed to scrape data for {link_text} in year {year}.")
                except Exception as e:
                    print(f"An error occurred while processing {link_text} in year {year}: {str(e)}")


In [ ]:
# Instantiate scheduler 
scheduler = ScrapeScheduler(table_for_links, range(2023, 2011, -1))

In [ ]:
# Scrape data to capture h1b data per company by year
scheduler.run_scrape_data_with_years(mode='normal')

In [ ]:
# We run this to capture missing company & year combos + combat lazy loading issues
scheduler.run_scrape_data_with_years(mode='missing', lazy_load = 720)

## Merge scraped csv files into combined dataframe and save as pickle 

In [ ]:
def combine_csv_files():
    dfs = []  # an empty list to store the data frames
    for filename in os.listdir('.'):  # '.' means current directory
        if filename.endswith('.csv'):
            try:
                dfs.append(pd.read_csv(filename))
            except EmptyDataError:
                print(f'Skipped empty file: {filename}')
                continue
    df = pd.concat(dfs, ignore_index=True)
    df.to_pickle("combined_data.pkl")  # Save the combined dataframe to a pickle file
    return df


In [ ]:
combined_df = combine_csv_files()

In [ ]:
import pandas as pd
import re

def get_filtered_dataframe(df: pd.DataFrame, keyword1: str, keyword2: str) -> pd.DataFrame:
    # create regex patterns
    keyword1_pattern = re.compile(keyword1, re.IGNORECASE)
    keyword2_pattern = re.compile(keyword2, re.IGNORECASE)

    # lower-case dataframe columns for comparison
    df_lower = df.applymap(lambda s:s.lower() if type(s) == str else s)

    # filter rows based on whether they contain either keyword
    matches_keyword1 = df_lower['Employer'].apply(lambda x: bool(keyword1_pattern.search(x)) if isinstance(x, str) else False) | df_lower['Job Title'].apply(lambda x: bool(keyword1_pattern.search(x)) if isinstance(x, str) else False)
    matches_keyword2 = df_lower['Employer'].apply(lambda x: bool(keyword2_pattern.search(x)) if isinstance(x, str) else False) | df_lower['Job Title'].apply(lambda x: bool(keyword2_pattern.search(x)) if isinstance(x, str) else False)

    # return slice of original df where either conditions is True
    return df[matches_keyword1 & matches_keyword2]

In [ ]:
get_filtered_dataframe(combined_df, keyword1 = "jp" ,keyword2 = "quant")

In [ ]:
combined_df